In [2]:
import nltk
from nltk.corpus import semcor
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer


In [7]:
nltk.download('all-corpora')

[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/abhishek/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/abhishek/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package bcp47 to
[nltk_data]    |     /home/abhishek/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/abhishek/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/abhishek/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/abhishek/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/abhishek/nltk_data...

True

In [70]:
!pip3 install num2words 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.5 MB/s eta 0:00:000:00:010:00:01
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=de84bdd7d99328acb113e907e06b057e134b2dfc66b85bb2ff3e3f8d67520535
  Stored in directory: /home/abhishek/.cache/pip/wheels/7c/d7/8d/2156234738063e3d4a39ba77dc677046100e62766b53807189
Successfully built docopt


## Preprocessing

In [78]:
from num2words import num2words
import numpy as np
import tqdm.notebook as tqdm
from string import punctuation
import math
from nltk.corpus import wordnet as wn



lemmatizer = WordNetLemmatizer()

EXTRA_STOPWORDS = ["''","'s","``"]
STOPWORDS = nltk.corpus.stopwords.words('english') + EXTRA_STOPWORDS
STOPWORDS += list(punctuation)

def treebank2wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('V'):
        return wn.VERB
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    else:
        return None
    

def syn2sense(syn):
    # get the sense (= lemma.postag.num) for a given synset
    s = syn.name()
    # n = ".".join(s.split(".")[-2:]) # n.01 and v.01 are different senses (eg: ash.n.01, ash.v.01)
    return s

def lemmatize(word,tag):
    if tag is None:
        return lemmatizer.lemmatize(word)
    else : 
        return lemmatizer.lemmatize(word,tag)
    
def num2Word(s):
    if s.isnumeric() and s.lower()!= "infinity" and s.lower()!="nan":
         s = num2words(s)
    return s

def clean(tokens):
    tagged = nltk.pos_tag(tokens)
    lemmatized = [lemmatize(w,treebank2wn(tag)) for w,tag in tagged]
    cleaned = [(num2Word(w), tag) for w, tag in zip(
        lemmatized, [tag for _, tag in tagged]) if w.lower() not in STOPWORDS]
    return cleaned



In [79]:
def parse(sent):
    tokens = []
    senses = []

    for e in sent:
        if isinstance(e, nltk.tree.Tree):
            lemma = e.label()
            if isinstance(lemma, nltk.corpus.reader.wordnet.Lemma):
                synset = lemma.synset()
                sense = syn2sense(synset)
            else:
                sense = None
            le = len(e)
            if le == 1:
                w = e[0]
                if isinstance(w, nltk.tree.Tree) or isinstance(w,list):
                    lw = len(w)
                    w = " ".join([w[i] for i in range(lw)])
            else:
                w = " ".join([e[i] for i in range(le)])
        elif isinstance(e, list):
            w = e[0]
            sense = None
        else:
            invtype = type(e)
            raise Exception("Invalid type: %s" % invtype)
        if w:
            tokens.append(w)
            senses.append(sense)
    return tokens, senses

In [80]:
semcor_data = semcor.tagged_sents(tag='sem')

X = []
y = []
words = []
for sent in semcor_data:
    try:
        tokens, senses = parse(sent)
        tagged_tokens = clean(tokens)
        
        for i in range(len(tokens)):
            context = tagged_tokens[max(
                0, i - 4):i] + tagged_tokens[i + 1:min(len(tagged_tokens), i + 5)]
            context_str = ' '.join(
                [w.lower() + '_' + tag for w, tag in context])
            X.append(context_str)
            y.append(senses[i])
    except Exception as e:
        print(e)




In [147]:
label_dict = {label: idx for idx, label in enumerate(set(y))}
y = [label_dict[label] for label in y]
print(y[0:10])

[16590, 15822, 7829, 21339, 16590, 2296, 16590, 206, 16590, 19458]


In [148]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()
X_ = vectorizer.fit_transform(X[:10000])
print(X_[0:10])
# replace None in Y with the correct variable
y_ = y[:10000]
# y_ = vectorizer.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)


MultinomialNB()

In [149]:
y_pred = nb_classifier.predict(X_test)


## Accuracy of model with local context + pos tags

In [150]:
print("Accuracy: ", nb_classifier.score(X_test, y_test))

Accuracy:  0.465


In [13]:

def clean(tokens):
    tagged = nltk.pos_tag(tokens)
    lemmatized = [lemmatize(w, treebank2wn(tag)) for w, tag in tagged]
    cleaned = [num2Word(w) for w in lemmatized if w.lower() not in STOPWORDS]
    return cleaned


In [75]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
import random 
# new Feature Space
# local context  : set of unordered words in the large context window
# topic context  : set of worods assigned with their positions in the  local context i.e. (word, position)
# collocation context : set of collocations of words i.e. (w-1w, w-2w-1w, ww1,ww1w2) like take a window of max size and get all the collocations from 2 to max size

semcor_data = semcor.tagged_sents(tag='sem')
vect = CountVectorizer(ngram_range=(1, 4))
X_new = []
y_new = []
words = []
cnt = 0
for sent in semcor_data:
    try:
        tokens, senses = parse(sent)
        tagged_tokens = clean(tokens)
        for i in range(len(tagged_tokens)):
            # take a window of 4 for words before and 4 for words after
            # another feature is to take a window of 4 words with possition before and after
            # another feature to take ngrams of 2 to 4
            f1 = tagged_tokens[max(0, i - 4):i] + tagged_tokens[i + 1:min(len(tagged_tokens), i + 5)]
            random.shuffle(f1)
            f2 = [tagged_tokens[i+j] + '_' + str(j) for j in range(-4, 5) if i + j >= 0 and i + j < len(tagged_tokens) and i != j]
            f2 = ' '.join(f2)
            f3 = []
            for l in range(2,4):
                temp = []
                for j in range(i-l,i+1):
                    if j>=0 and j+l<len(tagged_tokens):
                        temp.append(tagged_tokens[j:j+l+1])
                f3.extend(temp)
            f3 = [' '.join(x) for x in f3]
            # print(f3)
            # f3_bow = vect.fit_transform(f3)
            # f3_bow = ' '.join(vect.get_feature_names())
            # print(f3)
            context = f1 + ' ' + ' '.join(f2) + ' ' + ' '.join(f3)
            # print(context)
            X_new.append(context)
            y_new.append(senses[i])

    except Exception as e:
        print(e)


In [76]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

vectorizer = CountVectorizer()
# X_new_bow = vectorizer.fit_transform(X_new[:10000])
X_ = vectorizer.fit_transform(X_new[:10000])
label_dict = {label: idx for idx, label in enumerate(set(y_new))}
y_new = [label_dict[label] for label in y_new]
y_ = y_new[:10000]
X_train, X_test, y_train, y_test = train_test_split(
    X_, y_, test_size=0.2, random_state=42)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)



MultinomialNB()

In [77]:
y_pred = nb_classifier.predict(X_test)
print("Accuracy: ", nb_classifier.score(X_test, y_test))


Accuracy:  0.488


In [ ]:
# f1 = 47.45 
# f2 = 48.8 
# f3 = 12.45
# f1 + f2 = 48.8
